# PMR3508 - Trabalho 2: Aplicação de diferentes classificadores na base Adult

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

import sklearn
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler


In [ ]:
column_names = ["Id", "Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Marital Status",
        "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
        "Hours per week", "Country", "Income"]

In [ ]:
test_data = pd.read_csv("../input/test_data.csv", names = column_names, na_values='?').drop(0, axis = 0).reset_index(drop = True)
train_data = pd.read_csv("../input/train_data.csv", names = column_names, na_values='?').drop(0, axis = 0).reset_index(drop = True)

## Exploração e tratamento dos dados

 Podemos descartar as colunas 'Id' (sem utilidade para classificação) e 'Education' (informação já quantificada pela coluna Education-Num):

In [ ]:
train_data = train_data.drop("Id", axis = 1)
train_data = train_data.drop("Education", axis = 1)
test_data = test_data.drop("Id", axis = 1)
test_data = test_data.drop("Education", axis = 1)
test_data = test_data.drop("Income", axis = 1)

In [ ]:
test_data.head()

In [ ]:
train_data.columns

In [ ]:
train_data.shape

In [ ]:
train_data.head()

In [ ]:
train_data.info()

In [ ]:
train_data['Age'] = train_data['Age'].astype('int64')
train_data['fnlwgt'] = train_data['fnlwgt'].astype('int64')
train_data['Education-Num'] = train_data['Education-Num'].astype('int64')
train_data['Capital Gain'] = train_data['Capital Gain'].astype('int64')
train_data['Capital Loss'] = train_data['Capital Loss'].astype('int64')
train_data['Hours per week'] = train_data['Hours per week'].astype('int64')


In [ ]:
train_data.info()

In [ ]:
train_data.describe()

In [ ]:
train_data.describe(include=['object'])

In [ ]:
train_data = train_data.fillna('missing')  # Preencher dados faltantes com a string 'missing'
test_data = test_data.fillna('missing')

In [ ]:
def count_null_values(data):  # Returns a DataFrame with count of null values
    
    
    counts_null = []
    for column in data.columns:
        counts_null.append(data[column].isnull().sum())
    counts_null = np.asarray(counts_null)

    counts_null = pd.DataFrame({'feature': data.columns, 'count.': counts_null,
                                'freq. [%]': 100*counts_null/data.shape[0]}).set_index('feature', drop = True)
    counts_null = counts_null.sort_values(by = 'count.', ascending = False)
    
    return counts_null

In [ ]:
count_null_values(train_data)

Agora não há mais dados faltantes.

## Seleção de features

In [ ]:
train_data_analysis = train_data.apply(preprocessing.LabelEncoder().fit_transform)

In [ ]:
corr_mat = train_data_analysis.corr()
sns.set()
plt.figure(figsize=(15,12))
sns.heatmap(corr_mat, annot=True)

In [ ]:
train_data_analysis.corr().Income.sort_values()

In [ ]:
abs(train_data_analysis.corr().Income).sort_values(ascending=False)

Seleção dos atributos de maior correlação com a variável de interesse, os quais serão utilizados pelos classificadores:

In [ ]:
x_train = train_data[["Capital Gain", "Education-Num", "Relationship", "Age", "Hours per week", "Sex", "Marital Status", 
                      "Capital Loss"]].apply(preprocessing.LabelEncoder().fit_transform)
y_train = train_data.Income

x_test = test_data[["Capital Gain", "Education-Num", "Relationship", "Age", "Hours per week", "Sex", "Marital Status", 
                      "Capital Loss"]].apply(preprocessing.LabelEncoder().fit_transform)

In [ ]:
scaler = MinMaxScaler()  # Scaler para normalizar os dados contidos nos atributos

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

## Classificador: K-Nearest Neighbors

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 23, p = 1)
start = time.time()
scores = cross_val_score(knn, x_train, y_train, cv = 10)
print('K-Nearest Neighbors CV accuracy: {0:1.4f} +-{1:2.5f}\n'.format(scores.mean(), scores.std()))
print ('Time elapsed: {0:1.2f}\n'.format(time.time()-start))

In [ ]:
start = time.time()
knn.fit(x_train, y_train)
y_predict_knn = knn.predict(x_test)
print ('Time elapsed: {0:1.2f}\n'.format(time.time()-start))

## Classificador: Logistic Regression

In [ ]:
log_clf = LogisticRegression(solver = 'lbfgs', C = 1.0, penalty = 'l2', warm_start =  True)
start = time.time()
log_scores = cross_val_score(log_clf, x_train, y_train, cv = 10)
print('Logistic Regression CV accuracy: {0:1.4f} +-{1:2.5f}\n'.format(log_scores.mean(), log_scores.std()))
print ('Time elapsed: {0:1.2f}\n'.format(time.time()-start))

In [ ]:
start = time.time()
log_clf.fit(x_train, y_train)
y_predict_log = log_clf.predict(x_test)
print ('Time elapsed: {0:1.2f}\n'.format(time.time()-start))

## Classificador: Random Forest

In [ ]:
rf_clf = RandomForestClassifier(n_estimators = 700, max_depth = 12)
start = time.time()
rf_scores = cross_val_score(rf_clf, x_train, y_train, cv = 10)
print('Random Forest CV accuracy: {0:1.4f} +-{1:2.5f}\n'.format(rf_scores.mean(), rf_scores.std()))
print ('Time elapsed: {0:1.2f}\n'.format(time.time()-start))

In [ ]:
start = time.time()
rf_clf.fit(x_train, y_train)
y_predict_rf = rf_clf.predict(x_test)
print ('Time elapsed: {0:1.2f}\n'.format(time.time()-start))

## Classificador: Gaussian Naive Bayes

In [ ]:
gnb_clf = GaussianNB()
start = time.time()
gnb_scores = cross_val_score(gnb_clf, x_train, y_train, cv = 10)
print('Gaussian Naive Bayes CV accuracy: {0:1.4f} +-{1:2.5f}\n'.format(gnb_scores.mean(), gnb_scores.std()))
print ('Time elapsed: {0:1.2f}\n'.format(time.time()-start))

In [ ]:
start = time.time()
gnb_clf.fit(x_train, y_train)
y_predict_gnb = gnb_clf.predict(x_test)
print ('Time elapsed: {0:1.2f}\n'.format(time.time()-start))

In [ ]:
df_pred_knn = pd.DataFrame({'Income':y_predict_knn})
df_pred_log = pd.DataFrame({'Income':y_predict_log})
df_pred_rf = pd.DataFrame({'Income':y_predict_rf})
df_pred_gnb = pd.DataFrame({'Income':y_predict_gnb})

In [ ]:
df_pred_knn.to_csv("knn_prediction.csv", index = True, index_label = 'Id')
df_pred_log.to_csv("log_prediction.csv", index = True, index_label = 'Id')
df_pred_rf.to_csv("rf_prediction.csv", index = True, index_label = 'Id')
df_pred_gnb.to_csv("gnb_prediction.csv", index = True, index_label = 'Id')

## Considerações finais

Os classificadores utilizados neste <i>notebook</i> foram, como se pode verificar acima, <b>K-Nearest Neighbors</b>, <b>Logistic Regression</b>, <b>Random Forest</b> e <b>(Gaussian) Naive Bayes</b>. O primeiro foi incluído apenas para facilitar a comparação dos resultados obtidos no trabalho anterior. No entanto, nos limitamos à aplicação das técnicas <i>label encoding</i> e <i>scaling</i> dos atributos, sem no entanto realizar <i>grid search</i> devido ao longo tempo de processamento. Quando necessário, foram selecionado hiperparâmetros que garantem resultados considerados satisfatórios.

A ideia por trás deste trabalho foi comparar um pequeno conjunto de classificadores relativamente simples, comparáveis ao KNN, e ao mesmo tempo importantes; ou que partem de pressupostos que poderiam colocar sua eficácia em xeque num primeiro momento, como o <i>Naive Bayes</i> ou o <i>Random Forest</i>. Durante as aulas, foi enfatizado que, apesar da simplicidade, seus desempenhos são surpreendentes. Procurou-se verificar essa informação de maneira prática.

Não houve diferença significativa percebida na dificuldade de gerar os algoritmos de classificação, mas o mais simples foi o <i>Naive Bayes</i>, que não exigiu configuração de hiperparâmetros e apresentou um resultado razoável, mas que também apresentou a menor acurácia nos testes por <i>cross validation</i>, apesar da rápida execução.

Esses dois aspectos, relativa simplicidade e baixo tempo de execução combinados com altas acurácias, parecem explicar, ao menos em parte, por que esses algoritmos são considerados bem-sucedidos.

Quanto ao tempo de execução da validação cruzada, os mais demorados foram, respectivamente, o <i>Random Forest</i> (devido em grande parte aos hiperparâmetros selecionados) e o <i>K-Nearest Neighbors</i>, que também alcançaram os melhores resultados para este experimento. Os tempos para treino e realização da predição são consideravelmente encurtados, especialmente no caso do <i>Random Forest</i>.